In [ ]:
# Copyright (C) 2016-2018 by Jakob J. Kolb at Potsdam Institute for Climate
# Impact Research
#
# Contact: kolb@pik-potsdam.de
# License: GNU AGPL Version 3

%matplotlib inline

import numpy as np
import sympy as sp
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.stats import linregress

smallest_positive_number = np.finfo(dtype='float')

sp.init_printing()

input_data = 'res_raw.pkl'

Load right hand side of ode system

In [ ]:
rhs = np.load(input_data)

Define symbols for dynamic variables

In [ ]:
mucc, mucd, mudc, mudd = sp.symbols('mu_c^c mu_c^d mu_d^c mu_d^d', positive=True, real=True)
x, y, z, k = sp.symbols('x y z k')
c, g, p, g0 = sp.symbols('c, g, p, g_0')

Define symbols for parameters

In [ ]:
bc, bd, bR, g0, e, delta, rs, xi, p, k, epsilon, phi, tau, pi, N = sp.symbols('b_c b_d b_R g_0 e delta s xi p k epsilon phi tau, pi, N')

Set values for parameters and substitute them into the right hand side of the ode system

In [ ]:
N_val = 100
p_net = 0.125
alpha = 0.1
e_val = 100.
t_g = 100.
r_s = 0.23
b_c = 0.5
b_d = 1.2
d_c = 0.06
p_val = 5.
g_0 = 1.
params = {bc: b_c, bd: b_d, 
          bR: alpha**2 * e_val, 
          g0: g_0,
          e: e_val, delta: 0.06,
          rs: r_s, pi: 0.5,
          N: 1., p: p_val,
          xi: 1./4, p: p_val, 
          k: p_net * N_val, epsilon: 0.05, 
          phi: 0.8, tau: 1.}
variables = [x, y, z, mucc, mucd, mudc, mudd, c, g]

In [ ]:
rhs = rhs.subs(params)

In [ ]:
def dot_S(values, t):
    variables = [x, y, z, mucc, mucd, mudc, mudd, c, g]
    if values[-1] < alpha * g_0:
        values[-1] = alpha * g_0
    # add to g such that 1 - alpha**2 * (g/g_0)**2 remains positive
    subs1 = {var: val for (var, val) in zip(variables, values)}
    print t, 1 - alpha*(g_0/values[-1]), values[-1]
    return list(rhs.subs(subs1).evalf())

In [ ]:
t = np.linspace(0, 20, 100)
initial_conditions = [0, 0, 0, 1, 1, 1, 1, 1, g_0]
trajectory = odeint(dot_S, initial_conditions, t)

In [ ]:
res = {'parameters': params,
       'variables': variables,
       'initial conditions': initial_conditions,
       'trajectory': trajectory}

with open('analytic_trajectory.pkl', 'wb') as outf:
    pkl.dump(res, outf)

In [ ]:
slope = linregress(t, trajectory[:,8])
slope

In [ ]:
t_0 = -slope[1]/slope[0]

In [ ]:
t_0

In [ ]:
tmax = 40
plt.plot(t[0:-tmax], trajectory[0:-tmax,0:3])

In [ ]:
plt.plot(t, trajectory[:,3:7])

In [ ]:
plt.plot(t, trajectory[:,7])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(t[:], trajectory[:,8] - alpha)
